In [55]:
import pandas as pd
from google.cloud import bigquery

In [56]:
patent_count_query = '''
SELECT
  ids.company AS company_name,
  COUNT(p.url) AS patent_counts

FROM evident-data-dev.curated_taxonomies.company_ids AS ids

LEFT JOIN evident-data-dev.curated_google_patents.banking_patents AS p ON ids.id = p.company_id

WHERE ids.sector = 'bank'

GROUP BY company_name
'''

client = bigquery.Client(project="evident-data-dev")

C:\Users\lucas\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [57]:
new_patent_counts_df = client.query(patent_count_query).to_dataframe()

rename_new_dict  = {"company_name":"Company", "patent_counts":"New Patent Count"}
new_patent_counts_df.rename(columns=rename_new_dict, inplace=True)

C:\Users\lucas\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [61]:
index_patent_counts_df = pd.read_csv('2024_index_patent_data.csv', usecols=index_csv_cols)

rename_index_dict  = {"Company":"Company", "Raw Value\n2024":"2024 Index Patent Count"}
index_patent_counts_df.rename(columns=rename_index_dict, inplace=True)

In [72]:
patent_counts_df = pd.merge(new_patent_counts_df, index_patent_counts_df, on = "Company", how = "left")
patent_counts_df["Change"] = patent_counts_df["New Patent Count"] - patent_counts_df["2024 Index Patent Count"]

In [73]:
patent_counts_df

,Company,New Patent Count,2024 Index Patent Count,Change
0,Ally Financial,0,<NA>,<NA>
1,China Construction Bank,0,<NA>,<NA>
2,Bank of China,0,<NA>,<NA>
3,Agricultural Bank of China,0,<NA>,<NA>
4,Industrial and Commercial Bank of China,0,<NA>,<NA>
...,...,...,...,...
76,Citizens Financial Group,8,<NA>,<NA>
77,Charles Schwab,14,2,12
78,JPMorgan Chase,594,398,196
79,Capital One,1741,1376,365
